<a href="https://colab.research.google.com/github/joanjanku2000/ai_projects/blob/AI-1/Audio_Classification_(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataseti me audiot gjendet ne drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importimi i librarive te nevojshme per zgjidhjen e problemit

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.utils import to_categorical
import librosa

#Funksioni qe konverton audiot wav ne spectograma


In [ ]:
from os import listdir
from os.path import isfile, join

from matplotlib import pyplot as plt
import scipy.io.wavfile as wav


def wav_to_spectrogram(audio_path, save_path, spectrogram_dimensions=(256, 256), noverlap=16, cmap='gray_r'):

    sample_rate, samples = wav.read(audio_path)

    fig = plt.figure()
    fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.specgram(samples, cmap=cmap, Fs=2, noverlap=noverlap)
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    fig.savefig(save_path, bbox_inches="tight", pad_inches=0)


def dir_to_spectrogram(audio_dir, spectrogram_dir, spectrogram_dimensions=(256, 256), noverlap=16, cmap='gray_r'):

    file_names = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f)) and '.wav' in f]

    for file_name in file_names:
        print(file_name)
        audio_path = audio_dir + file_name
        spectogram_path = spectrogram_dir + file_name.replace('.wav', '.png')
        wav_to_spectrogram(audio_path, spectogram_path, spectrogram_dimensions=spectrogram_dimensions, 
                           noverlap=noverlap, cmap=cmap)

#Thirrja e funksionit te siper-definuar per pathet specifike ku gjendet dataseti si dhe direktorine target ku do te ruhen spektogramat

In [ ]:
audio_dir = "/content/drive/My Drive/Diploma_Data_Set_1/"
spectrogram_dir = "/content/drive/My Drive/Diploma_Data_Set_1/Spectograms/"
dir_to_spectrogram(audio_dir, spectrogram_dir)

#Importimi i imazheve prej pathit ku jane save-uar

In [ ]:
import glob
import random

images = glob.glob('/content/drive/My Drive/Diploma_Data_Set_1/Spectograms/*')
print(len(images))

## Ekstraktimi automatik i klasave nga dataseti ekzistues

### Marrja e emrave nga pathi ku gjenden imazhet dhe ruajtja e tyre ne matricat perkatese. Ndarja ne en/sq behet sipas fomatit qe jane audiot

In [ ]:
images_names = []
labels = []
for i in images:
  split = []
  digit = []
  split = i.split('/')
  images_names.append(split[6])
  digit = split[6].split('-')
  labels.append(digit[0])

In [ ]:
print(len(images_names))
print(len(labels))

260
260


## Ndarja e klasave unike qe gjenden, konkretisht 2 anglisht dhe shqip. Kjo me ndihmen e numpy -> unique function

In [ ]:
labels = np.array(labels)
classes = np.unique(labels)
nClasses = len(classes)
print('classes:{}'.format(classes))
print('Number of classes:{}'.format(nClasses))

classes:['en' 'sq']
Number of classes:2


In [ ]:
labels = labels.reshape(labels.shape[-1],1)

## Kategorizimi i labels (pra labels jane en- anglisht, sq - shqip)

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(labels)

print(labels.shape)

from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels)

print(labels.shape)

## Pergatitja e imazheve per t'u ushqyer ne rrjete

In [ ]:
from tensorflow.keras.preprocessing import image
from matplotlib.pyplot import imread
from tqdm import tqdm
img_width = 512
img_height = 512
X = []
for img_name in tqdm(images_names):
    image_path = '/content/drive/My Drive/Diploma_Data_Set_1/Spectograms/' + str(img_name)
    img = image.load_img(image_path,grayscale=True,target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)

Xtrain = np.array(X)

## Ndarja e te dhenave ne raportin 80% training data dhe 20% test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_labels, y_test_labels = train_test_split(Xtrain, labels, random_state = 0, test_size = 0.20)

In [ ]:
print(X_train.shape)
print(x_test.shape)

(208, 512, 512, 1)
(52, 512, 512, 1)


## Pergatitja e modelit

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nClasses, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
import keras
model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['binary_accuracy'])

In [ ]:
history = model.fit(X_train, Y_labels, batch_size=4, epochs=10, verbose=1)

In [ ]:
def plot_learningCurve(history, epoch):
  epoch_range = range(1, epoch+1)
  plt.plot(epoch_range, history.history['binary_accuracy'])
  # plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Saktesia e modelit')
  plt.ylabel('Saktesia')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  # plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Humbjet e modelit')
  plt.ylabel('Humbjet')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

plot_learningCurve(history, 10)

## Evaluimi i modelit me te dhena test (20% qe na mbeti)

In [ ]:
model.evaluate(x_test, y_test_labels)

2/2 [==============================] - 5s 3s/step - loss: 0.1499 - binary_accuracy: 0.9231


[0.14991140365600586, 0.9230769276618958]

In [ ]:
print(x_test.shape)
predictions = model.predict(x_test)

In [ ]:
y_pred = []
for i in range(0,len(predictions)):
  y_pred.append(np.argmax(predictions[i]))
y_pred = np.array(y_pred)
print(y_pred.shape)

(52,)


In [ ]:
y_classes = [np.argmax(y, axis=None, out=None) for y in y_test_labels]
y_classes = np.array(y_classes)
print(y_classes.shape)

(52,)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_classes, y_pred))
print(classification_report(y_classes, y_pred))

[[27  0]
 [ 0 25]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        25

    accuracy                           1.00        52
   macro avg       1.00      1.00      1.00        52
weighted avg       1.00      1.00      1.00        52

